# Fairness Metrics. 


---


## TODO:

  * For the selected data set, train at least one tree-based ensemble model (random forest, gbm, catboost or any other boosting)
  * For a selected variable (protected attribute, if possible choose sex or race, but if you do not have such variable then choose any categorical varaible) calculate selected fairness metrics (at least 3, like statistical parity, • equal opportunity, predictive equality),
  * train three or more candidate models (different variables, different transformations, different model structures) and compare fairness metrics between them.
  * Comment on the results for points (2) and (3)


## Data & preprocessing


---



For the sake of simplicity we'll re-use mushrooms dataset again, known already from the HW1, HW2 and HW3. Ordinnal encoder will be used, similarly to the HW3.

In [2]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv("mushrooms.csv")
encoder = OrdinalEncoder()

y = df.loc[:,['class']]
X = df.drop(['class'], axis=1)

y_temp = encoder.fit_transform(y)
X_temp = encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)

And we'll also need fairness metrics. Notation (based on the hiring environment for simplicity, as described in [here](https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb), though here we'll apply it to mushroom dataset):

  * X ∈ Rᵈ: quantified features of the applicant(e.g. education, work experience, college GPA, etc.).
  * A ∈ {0, 1}: a binary sensitive attribute(e.g. majority/minority).
  * C :=c(X,A) ∈ {0,1}: binary predictor (e.g. hire/reject), which makes decision based on a score R:=r(x, a) ∈ [0,1].
  * Y ∈ {0, 1}: target variable(e.g. if the candidate is truly capable of the position).
  * We assume X, A, Y are generated from an underlying distribution D i.e. (X, A, Y) ~ D.
  * We also denote P₀ [c] := P [c | A= 0].

In [95]:
# statistical parity
# P₀ [C=1] - P₁ [C=1]

def statisticalParity(data, predictions, protectedIndex, protectedValue):   
   protectedClass = [(x,l) for (x,l) in zip(data, predictions) 
      if x[protectedIndex] == protectedValue]   
   elseClass = [(x,l) for (x,l) in zip(data, predictions) 
      if x[protectedIndex] != protectedValue]
 
   if len(protectedClass) == 0 or len(elseClass) == 0:
      raise Exception("One of the classes is empty!")
   else:
      protectedProb = sum(1 for (x,l) in protectedClass if l == 1) / len(protectedClass)
      elseProb = sum(1 for (x,l) in elseClass  if l == 1) / len(elseClass)
 
   return elseProb, protectedProb


# equality of opportunity
# P₀ [C = 1| Y = 1] - P₁ [C = 1| Y = 1]

def equalOpportunity(data, predictions, labels, protectedIndex, protectedValue):   
   protectedClass = [(x,p,l) for (x,p,l) in zip(data, predictions, labels) 
      if x[protectedIndex] == protectedValue and l == 1]   
   elseClass = [(x,p,l) for (x,p,l) in zip(data, predictions, labels) 
      if x[protectedIndex] != protectedValue and l == 1]
 
   if len(protectedClass) == 0 or len(elseClass) == 0:
      raise Exception("One of the classes is empty!")
   else:
      protectedProb = sum(1 for (x,p,l) in protectedClass if p == 1) / len(protectedClass)
      elseProb = sum(1 for (x,p,l) in elseClass  if p == 1) / len(elseClass)
 
   return elseProb, protectedProb

# predictive rate parity - true/false
# P₀ [Y = 1| C= 1] = P₁ [Y = 1| C= 1] / P₀ [Y = 0| C= 0] = P₁ [Y = 0| C= 0]

def predictiveRateParity(data, predictions, labels, protectedIndex, protectedValue, predictionValue):   
   protectedClass = [(x,p,l) for (x,p,l) in zip(data, predictions, labels) 
      if x[protectedIndex] == protectedValue and p == predictionValue]   
   elseClass = [(x,p,l) for (x,p,l) in zip(data, predictions, labels) 
      if x[protectedIndex] != protectedValue and p == predictionValue]
 
   if len(protectedClass) == 0 or len(elseClass) == 0:
      raise Exception("One of the classes is empty!")
   else:
      protectedProb = sum(1 for (x,p,l) in protectedClass if l == predictionValue) / len(protectedClass)
      elseProb = sum(1 for (x,p,l) in elseClass  if l == predictionValue) / len(elseClass)
 
   return elseProb, protectedProb

def calculateMetrics(data, predictions, labels, protectedIndex, protectedValue):
   s_par = statisticalParity(data, predictions, protectedIndex, protectedValue)
   e_opp = equalOpportunity(data, predictions, labels, protectedIndex, protectedValue)
   p_r_par_true = predictiveRateParity(data, predictions, labels, protectedIndex, protectedValue, 1)
   p_r_par_false = predictiveRateParity(data, predictions, labels, protectedIndex, protectedValue, 0)
   print(f"Protected probability and complement probability (statistical parity): {s_par}")
   print(f"Protected probability and complement probability (equal opportunity): {e_opp}")
   print(f"Protected probability and complement probability (predictive rate parity - true): {p_r_par_true}")
   print(f"Protected probability and complement probability (predictive rate parity - false): {p_r_par_false}")

## First model

---

For the tree-based model, let's take xgboost.

In [5]:
import xgboost

xgb = xgboost.XGBClassifier().fit(X_train, y_train)
print(f"\nModel score on test set: {xgb.score(X_test, y_test)}")
print(f"Predictions for the first 10 observations: {xgb.predict(X_test[:10])}")
print(f"True values for the first 10 observations: {y_test[:10].tolist()}")

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Model score on test set: 1.0
Predictions for the first 10 observations: [1. 0. 0. 0. 1. 1. 0. 1. 1. 0.]
True values for the first 10 observations: [[1.0], [0.0], [0.0], [0.0], [1.0], [1.0], [0.0], [1.0], [1.0], [0.0]]


Since mushroom dataset is fairly simple, our xboost model trained very well. Let's calculate fairness metrics and see how they behave.

In [96]:
calculateMetrics(X_test, xgb.predict(X_test), y_test, 9, 0)

Protected probability and complement probability (statistical parity): (0.42185792349726775, 0.5957746478873239)
Protected probability and complement probability (equal opportunity): (1.0, 1.0)
Protected probability and complement probability (predictive rate parity - true): (1.0, 1.0)
Protected probability and complement probability (predictive rate parity - false): (1.0, 1.0)


## Second model

---

Next two models appeared in one of the previous homeworks. The idea was to try overly simple and overly complicated DNN architectures.

In [94]:
# And now, for the last point, let's re-use some code from HW1 to train simple NN classifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

model = Sequential()
#model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=16, epochs=100, validation_data=(X_val, y_val))

score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
325/325 [==============================] - 1s 2ms/step - loss: 1.1691 - accuracy: 0.5588 - val_loss: 1.0178 - val_accuracy: 0.5508
Epoch 2/100
325/325 [==============================] - 0s 1ms/step - loss: 1.0331 - accuracy: 0.5476 - val_loss: 0.9457 - val_accuracy: 0.5531
Epoch 3/100
325/325 [==============================] - 0s 1ms/step - loss: 0.9742 - accuracy: 0.5636 - val_loss: 0.8995 - val_accuracy: 0.5769
Epoch 4/100
325/325 [==============================] - 0s 1ms/step - loss: 0.9299 - accuracy: 0.5863 - val_loss: 0.8601 - val_accuracy: 0.5969
Epoch 5/100
325/325 [==============================] - 0s 1ms/step - loss: 0.8911 - accuracy: 0.6134 - val_loss: 0.8249 - val_accuracy: 0.6238
Epoch 6/100
325/325 [==============================] - 0s 1ms/step - loss: 0.8556 - accuracy: 0.6363 - val_loss: 0.7919 - val_accuracy: 0.6362
Epoch 7/100
325/325 [==============================] - 0s 1ms/step - loss: 0.8228 - accuracy: 0.6545 - val_loss: 0.7625 - val_accuracy: 0.6531

Fairness metrics for the second model:

In [102]:
calculateMetrics(X_test, (model.predict(X_test) > 0.5).astype(int), y_test, 9, 0)

Protected probability and complement probability (statistical parity): (0.3901639344262295, 0.5422535211267606)
Protected probability and complement probability (equal opportunity): (0.9248704663212435, 0.8416075650118203)
Protected probability and complement probability (predictive rate parity - true): (1.0, 0.9246753246753247)
Protected probability and complement probability (predictive rate parity - false): (0.9480286738351255, 0.7938461538461539)


## Third model

---

In [104]:
# And now, for the last point, let's re-use some code from HW1 to train simple NN classifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam

model_2 = Sequential()
model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(32, activation='relu'))
model_2.add(Dropout(rate=0.25))
model_2.add(Dense(1, activation='sigmoid')) 

model_2.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
history = model_2.fit(X_train, y_train, batch_size=16, epochs=30, validation_data=(X_val, y_val))

score = model_2.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
325/325 [==============================] - 2s 3ms/step - loss: 0.5944 - accuracy: 0.6726 - val_loss: 0.4213 - val_accuracy: 0.8169
Epoch 2/30
325/325 [==============================] - 1s 3ms/step - loss: 0.4431 - accuracy: 0.8007 - val_loss: 0.3037 - val_accuracy: 0.8900
Epoch 3/30
325/325 [==============================] - 1s 3ms/step - loss: 0.3413 - accuracy: 0.8615 - val_loss: 0.2256 - val_accuracy: 0.9254
Epoch 4/30
325/325 [==============================] - 1s 2ms/step - loss: 0.2927 - accuracy: 0.8886 - val_loss: 0.1808 - val_accuracy: 0.9377
Epoch 5/30
325/325 [==============================] - 1s 3ms/step - loss: 0.2493 - accuracy: 0.9023 - val_loss: 0.1593 - val_accuracy: 0.9415
Epoch 6/30
325/325 [==============================] - 1s 3ms/step - loss: 0.2120 - accuracy: 0.9154 - val_loss: 0.1215 - val_accuracy: 0.9538
Epoch 7/30
325/325 [==============================] - 1s 3ms/step - loss: 0.1744 - accuracy: 0.9334 - val_loss: 0.0931 - val_accuracy: 0.9677
Epoch 

And fairness metrics for the third model.

In [105]:
calculateMetrics(X_test, (model_2.predict(X_test) > 0.5).astype(int), y_test, 9, 0)

Protected probability and complement probability (statistical parity): (0.42185792349726775, 0.5957746478873239)
Protected probability and complement probability (equal opportunity): (1.0, 1.0)
Protected probability and complement probability (predictive rate parity - true): (1.0, 1.0)
Protected probability and complement probability (predictive rate parity - false): (1.0, 1.0)


## Comment

---

Although statistical parity shown some variation among all models, both XGB and overly-complicated DNN had a perfect equal opportunity and predictive rate parity. It might've been caused by the simplicity of our dataset - these 2 models reached 100% accuracy. Interrestingly, lower prediction confidence of simple-DNN led to worse fairness metrics.

As mushroom dataset doesn't have any natural candidates for protected values, I chose feature #9 - it had decently balanced distributioin of 0's vs other values.